Preprocess data to make it look/act better!

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
import pandas as pd
import numpy as np
import os

In [ ]:
# # Create paths
input_path = '/content/drive/MyDrive/FinalProject/sPINN//data/days45/original.csv'
output_path = '/content/drive/MyDrive/FinalProject/sPINN//data/days45/updated.csv'

# Load data
df = pd.read_csv(input_path)

# update datetime column(s)
df["datetime"] =pd.to_datetime(
    df[["#YY", "MM", "DD", "hh", "mm"]].rename(
        columns={
            "#YY": "year",
            "MM": "month",
            "DD": "day",
            "hh": "hour",
            "mm": "minute"
        }
    )
)

In [ ]:
# Cyclical time encoding
minutes = df["datetime"].dt.hour * 60 + df["datetime"].dt.minute

# Shift phase so peak occurs at midnight (θ + π/2)
theta = 2 * np.pi * (minutes / 1440)
df["time_sin"] = np.sin(theta + np.pi/2)

In [ ]:
# Compass directions converted to degrees
dir_map = {
    "N":0, "NNE":22.5, "NE":45, "ENE":67.5,
    "E":90, "ESE":112.5, "SE":135, "SSE":157.5,
    "S":180, "SSW":202.5, "SW":225, "WSW":247.5,
    "W":270, "WNW":292.5, "NW":315, "NNW":337.5
}

In [ ]:
df["SwD_deg"] = df["SwD"].map(dir_map)
df["WWD_deg"] = df["WWD"].map(dir_map)

df["SwD_sin"] = np.sin(np.deg2rad(df["SwD_deg"]))
df["SwD_cos"] = np.cos(np.deg2rad(df["SwD_deg"]))
df["WWD_sin"] = np.sin(np.deg2rad(df["WWD_deg"]))
df["WWD_cos"] = np.cos(np.deg2rad(df["WWD_deg"]))


df = df.drop(columns=["#YY","MM","DD","hh","mm","SwD","WWD", "SwD_deg", "WWD_deg"])

# Update Ordering
cols = df.columns.tolist()
cols.remove("datetime")
cols.remove("time_sin")
new_order = ["datetime", "time_sin"] + cols
df_final = df[new_order]

In [ ]:
# Save df
os.makedirs(os.path.dirname(output_path), exist_ok=True)
df_final.to_csv(output_path, index=False)

df_final.head()

,datetime,time_sin,WVHT,DPD,APD,MWD,WTMP,SwH,SwP,WWH,WWP,SwD_sin,SwD_cos,WWD_sin,WWD_cos
0,2025-12-02 23:26:00,0.989016,1.4,8.0,5.5,152.0,15.3,0.1,10.5,1.4,7.7,0.382683,-0.923880,0.382683,-0.923880
1,2025-12-02 22:56:00,0.961262,1.6,8.0,5.8,155.0,15.3,0.2,10.5,1.5,8.3,0.382683,-0.923880,0.382683,-0.923880
2,2025-12-02 22:26:00,0.917060,1.4,8.0,5.4,152.0,15.3,0.1,10.5,1.4,8.3,0.707107,-0.707107,0.382683,-0.923880
3,2025-12-02 21:56:00,0.857167,1.8,8.0,5.8,145.0,15.3,0.1,10.5,1.8,8.3,0.707107,-0.707107,0.707107,-0.707107
4,2025-12-02 21:26:00,0.782608,1.5,7.0,5.4,149.0,15.3,0.1,10.5,1.5,7.1,0.707107,-0.707107,0.382683,-0.923880
